In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install transformers 

     |████████████████████████████████| 2.5MB 31.1MB/s 
     |████████████████████████████████| 3.3MB 37.0MB/s 
     |████████████████████████████████| 901kB 41.5MB/s 


In [3]:
import torch
import pandas as pd
import numpy as np

# Thu vien transformer cho Classification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, RobertaForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification, XLMRobertaForSequenceClassification, RobertaForSequenceClassification

# Xu ly label
from sklearn.preprocessing import LabelEncoder

# Metric danh gia 
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

# Ve do thi
import seaborn as sn
import matplotlib.pyplot as plt

In [4]:
# Xay dung data de fit vao mo hinh 
class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [5]:
path = "/content/drive/MyDrive/Scocial Media Data Mining/Dataset"

In [6]:
train = pd.read_csv(path + "/UIT-ViCTSD/UIT-ViCTSD_train.csv", index_col=False)
dev = pd.read_csv(path + "/UIT-ViCTSD/UIT-ViCTSD_valid.csv", index_col=False)
test = pd.read_csv(path + "/UIT-ViCTSD/UIT-ViCTSD_test_text.csv", index_col=False)

In [7]:
%cd "/content/drive/MyDrive/Scocial Media Data Mining/SourceCode/Tu_PhoBert/UIT-ViCTSD/Constructiveness"

/content/drive/.shortcut-targets-by-id/1CZcWZiWtxpQUuZayUBOYLquIpN5fLP1b/Scocial Media Data Mining/SourceCode/Tu_PhoBert/UIT-ViCTSD/Constructiveness


In [8]:
X_train = train['Comment']
y_train = train['Constructiveness'].values

X_dev = dev['Comment']
y_dev = dev['Constructiveness'].values

#X_test = test['Comment']
#y_test = test['Constructiveness'].values

In [9]:
# encoding label 
le = LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [10]:
num_labels = len(np.unique(y_train))

In [11]:
path_model = ""

In [12]:
phobert = RobertaForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels = num_labels)
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
train_encodings = phobert_tokenizer(X_train.to_list(), truncation=True, padding=True, max_length=40)
dev_encodings = phobert_tokenizer(X_dev.to_list(), truncation=True, padding=True, max_length=40)
#test_encodings = phobert_tokenizer(X_test.to_list(), truncation=True, padding=True, max_length=40)

y_train_encoding = le.transform(y_train)
y_dev_encoding = le.transform(y_dev)
#y_test_encoding = le.transform(y_test)

In [14]:
# Chuan bi data

train_dataset = BuildDataset(train_encodings, y_train_encoding)
dev_dataset = BuildDataset(dev_encodings, y_dev_encoding)
#test_dataset = BuildDataset(test_encodings, y_test_encoding)

In [15]:
# Chuan bi mo hinh

training_args = TrainingArguments(
    output_dir='./results_1',          
    num_train_epochs=30,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=32,   
    warmup_steps=500,                
    weight_decay=0.01,
    no_cuda=False
)

phobert_trainer = Trainer(
    model=phobert,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=dev_dataset             
)

In [16]:
phobert_trainer.train()

Step,Training Loss
500,0.451600
1000,0.245400
1500,0.089700
2000,0.045500
2500,0.021600
3000,0.017700
3500,0.012600
4000,0.006900
4500,0.008700
5000,0.008900


TrainOutput(global_step=6570, training_loss=0.06964087667031557, metrics={'train_runtime': 1871.9101, 'train_samples_per_second': 112.185, 'train_steps_per_second': 3.51, 'total_flos': 6803990424000000.0, 'train_loss': 0.06964087667031557, 'epoch': 30.0})

In [22]:
X_test = test['Comment']
test['Constructiveness'] = 0
y_test = test['Constructiveness'].values #Gán cho y_test giá trị constant= 0 để đủ 2 cột đưa vào hàm
test_encodings = phobert_tokenizer(X_test.to_list(), truncation=True, padding=True, max_length=40)
y_test_encoding = le.transform(y_test)
test_dataset = BuildDataset(test_encodings, y_test_encoding)

In [24]:
# Du doan 
y_pred_classify = phobert_trainer.predict(test_dataset)

y_pred = np.argmax(y_pred_classify.predictions, axis=-1)

In [26]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
# Danh gia mo hinh
y_true = y_test_encoding
y_pred_classify = phobert_trainer.predict(test_dataset)
y_pred = np.argmax(y_pred_classify.predictions, axis=-1)
cf = confusion_matrix(y_true, y_pred, labels= pd.Series(y_true).unique())

print(cf)
print("accuracy: " + str(accuracy_score(y_true, y_pred)))
print("F1 - macro: " + str(f1_score(y_true, y_pred, average='macro')))
target_names = pd.Series(y_test).unique()
print('Classification report in Training set')


[[654]]
accuracy: 0.654
F1 - macro: 0.3954050785973398
Classification report in Training set


In [28]:
import pandas as pd

pd.DataFrame({"Y_Prediced": y_pred}).to_csv('PhoBert_ViCTSD_Constructiveness.csv', index= False)